<a href="https://colab.research.google.com/github/koleshjr/ALL_MY_TEMPLATES/blob/main/PseudoLabelling_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Rewritten code with pseudolabelling

model = CatBoostClassifier(
    n_estimators=10000,
    learning_rate=0.01124,
    auto_class_weights='Balanced',
    # class_weight='balanced'
)

skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
oof = np.zeros(len(train_df))
predictions = []

for fold, (trn_idx, val_idx) in enumerate(skfold.split(X, y)):
    print(f'Fold {fold + 1}')
    X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    
    # Pseudolabelling
    # Get the model predictions on the test set
    test_preds = model.predict_proba(test_df[X.columns])[:, 1]
    # Get the most confident predictions
    confident_preds_high = np.where(test_preds > 0.95)
    confident_preds_low = np.where(test_preds < 0.05)
    # Add these predictions to the X_train and y_train
    X_pseudo_high = test_df.iloc[confident_preds_high[0], :]
    X_pseudo_low = test_df.iloc[confident_preds_low[0], :]
    X_pseudo = pd.concat([X_pseudo_high, X_pseudo_low])
    y_pseudo_high = test_preds[confident_preds_high[0]]
    y_pseudo_low = 1 - test_preds[confident_preds_low[0]]
    y_pseudo = np.concatenate([y_pseudo_high, y_pseudo_low])
    X_train = pd.concat([X_train, X_pseudo])
    y_train = np.concatenate([y_train, y_pseudo])
    
    model.fit(
        X_train, y_train,
        eval_set=(X_valid, y_valid),
        verbose=100,
        early_stopping_rounds=250,
    )

    oof[val_idx] = model.predict(X_valid)
    predictions.append(model.predict_proba(test_df[X.columns])[:, 1])

predictions = np.mean(predictions, axis=0)
print(f'Our oof f1 score is {f1_score(y, oof)}')